# LLM Market Analytics Examples

This notebook demonstrates how to use the LLM-based market forecaster for analyzing and predicting market movements.

## Load and Prepare Sample Data

Let's create some sample market data for demonstration purposes.

In [3]:
from quantlib.datasource.service import DataService
data_service = DataService.create_default_service()

# Define parameters for data fetching
symbol = "PETR4"  # Change to your desired symbol
start_date = "2021-01-01"  # Adjust as needed
end_date = "2025-05-16"    # Adjust as needed
interval = "1m"            # 1-minute data

In [10]:
ts_object = data_service.get_close_prices(
        symbol=symbol,
        source='mt5',
        start_date=start_date,
        end_date=end_date,
        interval=interval
    )

In [15]:
ts_object.data.to_parquet('petr_returns.parquet')

## Use the data and build on unix enviroment

In [8]:
import pandas as pd
import numpy as np
import torch
from quantlib.analytics.llm import LLMForecaster, MarketTokenizer
from quantlib.data.timeseries import TimeSeries

In [2]:
ts = TimeSeries(pd.read_parquet('quantlib/notebooks/win_returns.parquet'))

In [3]:
ret = ts.returns(intraday_only=True)
returns = ret.data[(ret.data.index.hour >= 10) & (ret.data.index.hour < 17)]

In [4]:
# 2. Initialize the forecaster
forecaster = LLMForecaster()


In [13]:
# 3. Prepare the dataset
forecaster.build_dataset(returns['close'])

In [14]:
# 4. Build and train the model
forecaster.build_model()
forecaster.train()

Epoch 1, Loss: 0.1994
Epoch 2, Loss: 0.0354
Epoch 3, Loss: 0.0344
Epoch 4, Loss: 0.0341
Epoch 5, Loss: 0.0340


In [26]:
# 5. Generate predictions
# Create a prompt from recent market data
recent_data = returns['close'].iloc[-60:].values
prompt = torch.tensor(
    MarketTokenizer.series_to_tokens(
        pd.Series(recent_data),
        forecaster.vocab_size
    )[0],
    dtype=torch.long
).unsqueeze(0)  # Add batch dimension

# Generate predictions
predicted_tokens = forecaster.fit(prompt, steps=5)  # Predict next 5 days

# Convert predictions back to returns
predicted_returns = MarketTokenizer.tokens_to_values(
    np.array(predicted_tokens),
    forecaster.bins
)

print("Predicted returns for next periods:", predicted_returns)

Predicted returns for next 5 days: [ 2.67594612e-03  7.92015515e-03  7.29085006e-03 -5.08548325e-03
  7.92015515e-03 -1.32664493e-02  3.30525120e-03  4.98339809e-03
  4.14432464e-03 -1.09988439e-03  5.61270317e-03 -1.93895783e-03
 -1.09988439e-03  3.30525120e-03 -5.71478833e-03 -1.09988439e-03
 -1.32664493e-02 -2.77803128e-03  3.68494142e-04  4.98339809e-03
 -5.71478833e-03  9.97799226e-04 -1.93895783e-03  7.29085006e-03
 -1.93895783e-03  4.98339809e-03  1.02276071e-02  1.02276071e-02
 -4.70579302e-04 -4.24640980e-03 -1.09988439e-03  4.14432464e-03
 -3.40733636e-03 -2.77803128e-03  7.29085006e-03  5.61270317e-03
  2.67594612e-03 -4.70579302e-04 -5.08548325e-03 -8.02224031e-03
  2.67594612e-03  1.10666806e-02 -1.09988439e-03 -4.70579302e-04
 -2.77803128e-03 -1.93895783e-03  1.29545958e-02  9.97799226e-04
 -1.09988439e-03 -1.09988439e-03 -9.49061883e-03 -1.93895783e-03
  1.29545958e-02  3.30525120e-03  3.68494142e-04  1.83687267e-03
 -5.10425799e-05 -5.10425799e-05 -5.10425799e-05 -5.104

In [27]:
# 6. Evaluate predictions (example with simple metrics)
actual_returns = returns.iloc[-5:].values
mse = np.mean((predicted_returns - actual_returns) ** 2)
print(f"Mean Squared Error: {mse:.6f}")

Mean Squared Error: 0.000031
